In [1]:
import time
from tqdm import tqdm
from multiprocessing import Pool
from sklearn import preprocessing
from obspy.signal.headers import clibsignal
import math

import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, SecondLocator, DateFormatter
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import obspy as op
from obspy import read,read_inventory,read_events, UTCDateTime, Stream, Trace
from obspy.clients.fdsn.client import Client
from obspy.signal.rotate import rotate_ne_rt
from obspy.geodetics import gps2dist_azimuth,kilometers2degrees
from obspy.taup import TauPyModel

import json
import glob
import os
import numpy as np
from itertools import combinations
import pandas as pd

import pyarrow.feather as feather

import datetime

____________
# Setup
____________

In [2]:
from parameters_py.config import (
					WAVEFORM_DIR,CATALOG_FILE,XML_DIR,ORIENTATION_OUTPUT
				   )

Reading configuration file: ./config_file.cnf


In [3]:
ORIENTATION_OUTPUT

'/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/OUTPUT/'

---
# Functions

In [4]:
from src.analyse import (
					aic_simple,find_orientation,Braunmiller_Pornsopin_algorithm,calculate_metrics
				   )

from src.utils import (
					quakeml_to_dataframe,moment_tensor_to_nodal_planes,calculate_plunge,mecclass,adjust_baz_for_ZEN,rms,energy
				   )


---
# Main program
---

In [5]:
print('===============')
print('Reading catalog')
print('===============')
print('\n')

cat = quakeml_to_dataframe(CATALOG_FILE)
cat.tail(2)

Reading catalog




FileNotFoundError: [Errno 2] No such file or directory: '/home/sysop/dados_posdoc/RSBR_15_years/obspyDMT/CATALOG/CMTSOLUTIONS_2010_2025.xml'

In [ ]:
print('================')
print('Reading stations')
print('================')
print('\n')

STATIONS_xml = sorted(glob.glob(XML_DIR+'*BR.CZ*'))
print('Number of stations:',len(STATIONS_xml))

In [ ]:
STATIONS_xml

In [ ]:
def calculate_orientation(input_lst):

    xml_file = input_lst[0]
    wave_dir = input_lst[1]
    
    # ---------------
    # Read XML file
                        
    station_xml = op.read_inventory(xml_file)
    network = station_xml[0].code
    station = station_xml[0][0].code    
    
    # ---------------------------
    # Retrieving events waveforms
    # ---------------------------
    
    for evid,event in tqdm(cat.iterrows(), total=len(cat),desc=station+' orientation'):
        # ------------------------------
        # Check if the event is eligible

        evdp = event['depth']
        evmag = event['mag']
        evtype = event['magType']
        evtime = UTCDateTime(event['time'])
        evname = UTCDateTime(event['time']).strftime('%Y.%j.%H.%M.%S')
        evname2 = UTCDateTime(event['time']).strftime('%Y%m%d_%H%M%S')

        year = UTCDateTime(event['time']).strftime('%Y')
        julian_day = UTCDateTime(event['time']).strftime('%j')
                            
        # Epicentral distance:
            
        stlo = station_xml[-1][-1][-1].longitude
        stla = station_xml[-1][-1][-1].latitude
            
        evla = event['latitude']
        evlo = event['longitude']
            
        dist,az,baz = gps2dist_azimuth(evla, evlo,stla, stlo)
        gcarc = kilometers2degrees(dist/1000)
        
        # Taup: theoretical travel times 
        model = TauPyModel(model="iasp91")
        arrivals = model.get_travel_times(source_depth_in_km=evdp,distance_in_degree=gcarc,phase_list=['P','PKP','PKIKP'])
        
        if len(arrivals) > 0 and (gcarc < 100 or 140 < gcarc <= 180):
            
            # Event time + phase arrival time
            evtime = evtime+arrivals[0].time
            
            # ----------------------------
            # Check if feather file exists
            
            output_FEATHER_FILES_ORIENTATION = ORIENTATION_OUTPUT+'FEATHER_FILES/ORIENTATION/'+network+'.'+station+'/'
            
            file_feather_name = output_FEATHER_FILES_ORIENTATION+network+'.'+station+'.'+evname+'.orientation.feather'
    
            station_pwd = glob.glob(wave_dir + '/*')
   
            if os.path.isfile(file_feather_name):
                pass
        
            else:
                # -------------------------------
                # Check if components file exists
                        
                if 1 == 1:    

                    files = [x for x in station_pwd if x.endswith(evname2)]

                    if len(files) >= 3:
                        try:
                            file_HHE = files[component_index(files, "E")]
                            file_HHN = files[component_index(files, "N")]
                            file_HHZ = files[component_index(files, "Z")]

                        except:

                            file_HHE = files[component_index(files, "2")]
                            file_HHN = files[component_index(files, "1")]
                            file_HHZ = files[component_index(files, "Z")]

            
                        # # # --------
                        # # # Data HHE
                                
                        tr2_data_file = op.read(file_HHE)
                        tr2_data_file.trim(evtime-TIME_WINDOW,evtime+TIME_WINDOW)
                        tr2_data_file.taper(type='cosine',max_percentage=0.1)
                        tr2_data_file.filter('bandpass',freqmin=PERIOD_BANDS[0],freqmax=PERIOD_BANDS[1],zerophase=True, corners=4)
                    
                        # --------
                        # Data HHN
                    
                        tr1_data_file = op.read(file_HHN)
                        tr1_data_file.trim(evtime-TIME_WINDOW,evtime+TIME_WINDOW)
                        tr1_data_file.taper(type='cosine',max_percentage=0.1)
                        tr1_data_file.filter('bandpass',freqmin=PERIOD_BANDS[0],freqmax=PERIOD_BANDS[1],zerophase=True, corners=4)
                            
                        # --------
                        # Data HHZ
                                
                        trZ_data_file = op.read(file_HHZ)
                        trZ_data_file.trim(evtime-TIME_WINDOW,evtime+TIME_WINDOW)
                        trZ_data_file.taper(type='cosine',max_percentage=0.1)
                        trZ_data_file.filter('bandpass',freqmin=PERIOD_BANDS[0],freqmax=PERIOD_BANDS[1],zerophase=True, corners=4)

                        if len(tr2_data_file) > 0 and len(tr1_data_file) > 0 and len(trZ_data_file) > 0:
                               
                             if (tr2_data_file[0].stats.npts == tr1_data_file[0].stats.npts == trZ_data_file[0].stats.npts) and (trZ_data_file[0].stats.npts > TIME_WINDOW * 40):
                                
                                tr2_data_filtered = tr2_data_file[0].data[1000:-1000]
            
                                tr1_data_filtered = tr1_data_file[0].data[1000:-1000]
                                    
                                trZ_data_filtered = trZ_data_file[0].data[1000:-1000]
                                trZ_time = trZ_data_file[0].times()[1000:-1000]-TIME_WINDOW

                                # -------------------------------------------
                                # Function estimates the Akaike Information directly from data 
                                # The Summed Log Likelihood section implies that a natural 
                                # changepoint estimate is the sample index that minimizes 
                                # the AIC in equation
                                
                                aic_curve = aic_simple(trZ_data_filtered)
                                
                                k_min_index = np.argmin(aic_curve)
                            
                                time_P_arr = trZ_time[k_min_index]
                                                        
                                # Time error:
                                time_ins = round(time_P_arr,1)
                            
                                if time_ins > 0:
                                    signal_window_start = time_ins
                                    signal_window_final = time_ins+TIME_FINAL_P
                                    noise_window_start = time_ins
                                    noise_window_final = time_ins-TIME_FINAL_P
                                else:
                                    signal_window_start = time_ins
                                    signal_window_final = TIME_FINAL_P+time_ins
                                    noise_window_start = time_ins
                                    noise_window_final = -(abs(time_ins)+TIME_FINAL_P) 
                                # -------------------------------------------------------------------------------------------------------------------------------
                                # Signal and noise windows
                                        
                                signal_window = (trZ_time >= signal_window_start) & (trZ_time <= signal_window_final)
                                noise_window = (trZ_time >= noise_window_final) & (trZ_time <= noise_window_start)
                                
                                noise = trZ_data_filtered[noise_window]
                                trZ_noise_time = trZ_time[noise_window]
            
                                tr2 = tr2_data_filtered[signal_window]
                                tr1 = tr1_data_filtered[signal_window]
                                trZ = trZ_data_filtered[signal_window]
                                trZ_signal_time = trZ_time[signal_window]
                                    
                                # -------------------------------------------------------------------------------------------------------------------------------
                                # Calculating the optimal orientation
                                                    
                                results = Braunmiller_Pornsopin_algorithm(tr1,tr2,trZ,noise,baz,time_ins,CCVR_MIN=0.45,SNR_MIN=10,TRR_MIN=0.45,RVR_MIN=-1)

                                # -------------------------------------------------------------------------------------------------------------------------------
                                # Calculating the Plunge of: P, B, and T axis

                                nodal_planes = moment_tensor_to_nodal_planes(event['moment tensor'])

                                event_class = mecclass(nodal_planes)

                                # ----------------------------------------------------------------------------------------------------                               
                                # Creating a Pandas DataFrame:
                                column_info = [network,station,stla,stlo,evname,evla,evlo,evtime,evmag,evtype,evdp,dist,gcarc,baz,tr1_data_filtered,tr2_data_filtered,trZ_data_filtered,trZ_time,results['SS_best'],results['signal_strength'],results['SZR_best'],results['similarity_ZR'],results['ERTR_best'],results['energy_ratio_TR'],results['ERRZ_best'],results['energy_ratio_RZ'],results['SNR'],results['phi'],results['theta'],aic_curve,time_ins,results['quality'],results['gain_HHN'],results['gain_HHE'],results['gain_HHZ'],event['moment tensor'],nodal_planes,event_class]
                                columns_header = ['network','station','stla','stlo','evname','evla','evlo','evtime','evmag','evtype','evdp','distance','gcarc','baz','tr1_data','tr2_data','trZ_data','trZ_time','SS_best','signal_strength','SZR_best','similarity_vertical_radial','ERTR_best','energy_transverse_radial','ERRZ_best','energy_radial_vertical','SNR','phi','theta','aic_curve','clock_error','quality','gain_HHN','gain_HHE','gain_HHZ','moment tensor','nodal_planes','event_class']
                                orient_p_wave_df = pd.DataFrame(column_info, index=columns_header).T
                                orient_p_wave_df['evtime'] = pd.to_datetime(orient_p_wave_df['evtime'].apply(lambda x: x.isoformat() if isinstance(x, UTCDateTime) else x))

                                # ----------------------------------------------------------------------------------------------------
                                # Convert from pandas to Arrow and saving in feather formart file
                                os.makedirs(output_FEATHER_FILES_ORIENTATION,exist_ok=True)
                                feather.write_feather(orient_p_wave_df, file_feather_name)

# Creating input list for different folders

In [ ]:
input_list = []
for xml_f in STATIONS_xml:
    for wa_dir in WAVEFORM_DIR:
        input_list.append([xml_f,wa_dir])   

In [ ]:
input_list

In [ ]:
start_time = time.time()

with Pool(processes=num_processes) as p:
    max_ = len(STATIONS_xml)
    with tqdm(total=max_,) as pbar:
        for result in p.imap_unordered(calculate_orientation,input_list):
            pbar.update()

print('\n')
print("--- %.2f execution time (min) ---" % ((time.time() - start_time)/60))
print('\n')